This is a list of antimicrobial medicines. 

In [1]:
from ebmdatalab import bq
import os
import pandas as pd
import numpy as np

#mapping = pd.read_csv('../lib/ingredients.csv')
#cats = pd.read_csv('../lib/categories.csv', header = None, names=["category_code","category"])

#mapping.groupby('product_id')["category_id"].nunique()
#mapping['product_id'].nunique()

#mapping = mapping.join(cats, on='product_id', how="left")



## Antimicrobials in BNF

In [2]:
sql = '''
WITH bnf_codes AS (  
  SELECT presentation_code as bnf_code, 
  IF(para IN ('Some other antibacterials','Antituberculosis drugs', 'Antileprotic drugs'),
        'Others', para) AS para  
  FROM hscic.bnf 
  WHERE 
  (presentation_code LIKE '0501%')
) 
  
SELECT "vmp" AS type, vmp.id, vmp.bnf_code, vmp.nm, ing.nm AS ingredient, 
  route.descr as route, para as paragraph
FROM dmd.vmp 
INNER JOIN bnf_codes b ON b.bnf_code = vmp.bnf_code
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing
LEFT JOIN dmd.droute on vmp.id = droute.vmp	
LEFT JOIN dmd.route on route.cd = droute.route

ORDER BY type, nm  '''

antibac_meds = bq.cached_read(sql, csv_path=os.path.join('..','data','antibac_meds.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
antibac_meds[["id", "bnf_code"]].count()

antibac_meds["source"] = "bnf"

In [3]:
#antibac_meds.merge(mapping, left_on='id', right_on='product_id', how='left')
# check that each ingredient is mapped to just one paragraph
print(antibac_meds.groupby(['ingredient'])['paragraph'].nunique().max())
print("No. of paras: ", antibac_meds['paragraph'].nunique())
para_lookup = antibac_meds[['ingredient','paragraph']].drop_duplicates()

1
No. of paras:  11


## Dm+d Additions
There are some medicines used in hospital only, which don't have BNF codes (used for primary care). 

We will manually select these from dmd using the ingredients found in the products we identified from BNF codes.

In [4]:
  
# set name of codelist for exporting file
codelist_name="antibac_meds"

# import or paste list of vtms (Virtual Therapeutic Moieties / Ingredients) by name
names = antibac_meds["ingredient"].drop_duplicates()
print("No of antibiotic ingredients from bnf list: ", len(names))

# filter out ingredients identified as not being antimicrobial agents
names_filtered = [k for k in names if k not in ["Citric acid", "Thalidomide", "Sodium bicarbonate", "Sodium citrate"]]
print("No of antibiotic ingredients after filtering: ", len(names_filtered))

# convert list to tuple for use in SQL query
names_tuple = tuple(names_filtered)
if len(names_tuple)==1:
    # remove comma if only one item
    names_tuple = str(names_tuple).replace(",","")

sql = f'''
SELECT "vmp" AS type, vmp.id, bnf_code, vmp.nm, ing.nm AS ingredient, 
    route.descr as route, ddd.ddd, ddd.ddd_uomcd AS ddd_uomcd
FROM dmd.vmp
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm IN {names_tuple}
LEFT JOIN dmd.droute on vmp.id = droute.vmp  -- NB this introduces some duplicates we will remove later
LEFT JOIN dmd.route on route.cd = droute.route
LEFT JOIN dmd.ddd on vmp.id=ddd.vpid

ORDER BY type, nm  '''

dmd_antibac_meds = bq.cached_read(sql, csv_path=os.path.join('..','data',f'dmd_{codelist_name}.csv'))

print("No of antibiotic ingredients in dmd list: ", dmd_antibac_meds["ingredient"].nunique())

No of antibiotic ingredients from bnf list:  122
No of antibiotic ingredients after filtering:  118
No of antibiotic ingredients in dmd list:  118


In [5]:
# join BNF paragraph via ingredients
print("No of records before join:", len(dmd_antibac_meds["id"]))
dmd_antibac_meds_2 = dmd_antibac_meds.merge(para_lookup, on='ingredient', how='left')
print("No of records after join:",len(dmd_antibac_meds_2["id"]))

# some (3) multi-ingredient products map to different paragraphs (Genta + Clinda x2, Genta + Vancomycin). 
# We'll assign them to Other.
dmd_antibac_meds_2["para_count"] = dmd_antibac_meds_2.groupby("id")["paragraph"].transform('nunique')
#print(dmd_antibac_meds_2.loc[dmd_antibac_meds_2["para_count"]>1])
dmd_antibac_meds_2.loc[dmd_antibac_meds_2["para_count"]>1, "paragraph"] =  "Other"

dmd_antibac_meds_2.head()

No of records before join: 1045
No of records after join: 1045


,type,id,bnf_code,nm,ingredient,route,ddd,ddd_uomcd,paragraph,para_count
0,vmp,35899811000001104,0501040C0AAADAD,Amikacin 100mg/2ml solution for injection vials,Amikacin sulfate,Intramuscular,1.0,258682000.0,Aminoglycosides,1
1,vmp,35899811000001104,0501040C0AAADAD,Amikacin 100mg/2ml solution for injection vials,Amikacin sulfate,Intravenous,1.0,258682000.0,Aminoglycosides,1
2,vmp,33516711000001107,NaN,Amikacin 2.5% eye drops preservative free,Amikacin sulfate,Ocular,NaN,NaN,Aminoglycosides,1
3,vmp,35104511000001105,NaN,Amikacin 25mg/5ml solution for injection pre-filled syringes,Amikacin sulfate,Intrathecal,NaN,NaN,Aminoglycosides,1
4,vmp,35111911000001104,NaN,Amikacin 25mg/5ml solution for injection vials,Amikacin sulfate,Intrathecal,NaN,NaN,Aminoglycosides,1


**Note the above should have captured all the codes we previously found by selecting based on BNF codes so we don't need to join the tables.**

### Investigate routes of administration

In [6]:
dmd_antibac_meds_2.groupby("route")["id"].count().sort_values(ascending=False)

route
Oral                                      568
Intravenous                               193
Intramuscular                              69
Ocular                                     53
Cutaneous                                  43
Route of administration not applicable     30
Auricular                                  19
Inhalation                                 17
Intrathecal                                 7
Intraarticular                              5
Intracameral                                5
Intrapleural                                5
Intravitreal                                4
Intralesional                               3
Vaginal                                     3
Rectal                                      3
Intraperitoneal                             3
Nasal                                       2
Oromucosal                                  2
Gingival                                    2
Gastroenteral                               2
Subcutaneous                

In [7]:
# categorise routes of administration

condlist = [dmd_antibac_meds_2["route"]== "Oral",
            dmd_antibac_meds_2["route"].isin(["Intravenous", "Subcutaneous", "Intramuscular", "Intrapleural", 
                                        "Intraperitoneal", "Intraarticular", "Intracavernous", "Intrathecal",
                                        "Intracerebroventricular"]),
            dmd_antibac_meds_2["route"].isin(["Cutaneous", "Auricular","Vaginal","Intralesional",
                                        "Ocular", "Nasal", "Oromucosal", "Gingival", "Dental"]),
            dmd_antibac_meds_2["route"].isin(["Inhalation", "Intravesical", "Gastroenteral"])
                                        ]
choicelist = ["Oral", "Injectable", "Topical", "Other"]
dmd_antibac_meds_2["Route"] = np.select(condlist, choicelist)

# check for and correct vmps with multiple routes
dmd_antibac_meds_2 = dmd_antibac_meds_2.copy()
dmd_antibac_meds_2["Route_count"] = dmd_antibac_meds_2.groupby(["id"])["Route"].transform('nunique')
# most of those with mixed routes have "for injection" or "for infusion" in their name
dmd_antibac_meds_2.loc[(dmd_antibac_meds_2["Route_count"]>1) & (
                        (dmd_antibac_meds_2["nm"].str.contains("injection")) | (
                                    dmd_antibac_meds_2["nm"].str.contains("infusion"))), 
                                    "Route"] =  "Injectable"
#recount and assign any remaining to "Other" - this should be restricted to colistin                              
dmd_antibac_meds_2["Route_count"] = dmd_antibac_meds_2.groupby(["id"])["Route"].transform('nunique')
dmd_antibac_meds_2.loc[dmd_antibac_meds_2["Route_count"]>1, "Route"] =  "Other"

print(dmd_antibac_meds_2.loc[dmd_antibac_meds_2["Route_count"]>1][["id", "nm", "Route_count", "route", "Route"]])

dmd_antibac_meds_2 = dmd_antibac_meds_2.drop("route", axis=1)

                   id                                      nm  Route_count  \
320  4615111000001106  Colistin 19.5million unit powder vials            2   
321  4615111000001106  Colistin 19.5million unit powder vials            2   
322  4615111000001106  Colistin 19.5million unit powder vials            2   
323  4615111000001106  Colistin 19.5million unit powder vials            2   

         route  Route  
320       Oral  Other  
321  Auricular  Other  
322  Cutaneous  Other  
323     Ocular  Other  


In [8]:
# check for remaining dmds with multiple routes of administration
test = dmd_antibac_meds_2.copy()#[["id", "bnf_code","Route"]]
test["Route_count"] = test.groupby(["id"])["Route"].transform('nunique')

test = test.loc[test["Route_count"]>1]
print(test)

Empty DataFrame
Columns: [type, id, bnf_code, nm, ingredient, ddd, ddd_uomcd, paragraph, para_count, Route, Route_count]
Index: []


In [9]:
# check number of distinct medicines
print(dmd_antibac_meds_2["nm"].agg({"count", "nunique"}))
dmd_antibac_meds_2.groupby("type")["nm"].agg({"count", "nunique"})

nunique     897
count      1045
Name: nm, dtype: int64


,nunique,count
type,,
vmp,897,1045


In [10]:
# check how many products have DDDs
dmd_antibac_meds_2["ddd_flag"] = np.where(dmd_antibac_meds_2["ddd"].notnull(), 1, 0)
print(dmd_antibac_meds_2.groupby(["id"])["ddd"].agg({"nunique","count"}).max())
print(dmd_antibac_meds_2.groupby(["Route"])["ddd_flag"].agg({"sum","count"}))


count      5
nunique    1
dtype: int64
            sum  count
Route                 
0             3     44
Injectable  210    298
Oral        507    563
Other         8     15
Topical       3    125


In [13]:
dmd_antibac_meds_2 = dmd_antibac_meds_2.drop(["Route_count","para_count"], axis=1)

In [14]:
dmd_antibac_meds_2.to_csv(os.path.join('..','data','antibac_codelist.csv')) #export to csv here
